In [1]:
import tensorflow_addons as tfa
from utils.dataloader import *
from utils.dataset import *
from utils.layer import *
from utils.loss_function import *

2023-02-26 11:57:55.799970: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.keras.backend.clear_session()


def normalize(image):
    return dict(image=image / 255)


dataset_train = MNIST(transform=normalize)
dataset_valid = MNIST(False, transform=normalize)

batch_size = 128
epochs = 1

trainloader = DataLoader(dataset_train, batch_size, True)
validloader = DataLoader(dataset_valid, batch_size)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((28, 28, 1)))
model.add(tf.keras.layers.Conv2D(32, 5, use_bias=False))
model.add(tfa.layers.InstanceNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv2D(64, 3, use_bias=False))
model.add(tfa.layers.InstanceNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv2D(64, 3, use_bias=False))
model.add(tfa.layers.InstanceNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv2D(128, 3, 2, use_bias=False))
model.add(tfa.layers.InstanceNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(PrimaryCapsule(16, 8, depthwise=True, squash="self_attention"))
model.add(CapsuleSA(10, 16))
model.add(CapsuleLength())
model.summary()
model.compile(
    optimizer=tfa.optimizers.AdamW(
        1e-4,
        tf.keras.optimizers.schedules.ExponentialDecay(
            batch_size * 5e-4 / 16, len(trainloader) * epochs, 0.98
        ),
    ),
    loss=Margin(),
    metrics=["accuracy"],
)
history = model.fit(trainloader, validation_data=validloader, epochs=epochs)

2023-02-26 11:57:57.413940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 11:57:57.416003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 11:57:57.416166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 11:57:57.416511: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        800       
                                                                 
 instance_normalization (Ins  (None, 24, 24, 32)       64        
 tanceNormalization)                                             
                                                                 
 activation (Activation)     (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18432     
                                                                 
 instance_normalization_1 (I  (None, 22, 22, 64)       128       
 nstanceNormalization)                                           
                                                                 
 activation_1 (Activation)   (None, 22, 22, 64)        0

2023-02-26 11:58:00.486304: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-02-26 11:58:01.171942: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fbb9629a5e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-26 11:58:01.171974: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2023-02-26 11:58:01.221852: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


469/469 [==============================] - 10s 16ms/step - loss: 0.0696 - accuracy: 0.9432 - val_loss: 0.0156 - val_accuracy: 0.9897
